Original data:

**Table:** Person

| id | email            |
|----|------------------|
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |


In [ ]:
import pandas as pd
from pandasql import sqldf  # this is for making the sql querys within the jupyter notebook. 

I want to run SQL commands also, so:

In [50]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Create the Dataframe:

In [65]:
# Create the Person DataFrame to play with:

Person = {'id': [1,2,3],
        'email': ['john@example.com', 'bob@example.com', 'john@example.com']}

Person = pd.DataFrame(Person)
Person2 = pd.DataFrame(Person)

This is the Python Colution:

In [52]:
Person1 = Person.drop_duplicates(subset='email', keep='first')

Show Solution:

In [53]:
Person1

,id,email
0,1,john@example.com
1,2,bob@example.com


Now, the SQL Solution. we will build a simple query:

1. Select od pver emails and order by email, so we will have several ids per mail. 
2. then, we select the id = 1 to keep only the first row. 
3. Then we erase the others!

In [75]:
%sql sqlite:///Person2.db 
# to use the data base. 

In [82]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
''' I want to do this:  """
DELETE FROM Person2
WHERE id NOT IN (
    SELECT id
    FROM (
        SELECT id, ROW_NUMBER() OVER (PARTITION BY email ORDER BY id) AS row_num
        FROM Person2
    ) AS ranked
    WHERE row_num = 1
);
"""
'''

# but the tools only make you select and find things, but we can rename the selection:

query2 = """
SELECT *
FROM Person2
WHERE id IN (
    SELECT id
    FROM (
        SELECT id, ROW_NUMBER() OVER (PARTITION BY email ORDER BY id) AS row_num
        FROM Person2
    ) AS ranked
    WHERE row_num = 1
);
"""

In [88]:
df_EliDup = pysqldf(query2)

In [89]:
print(df_EliDup)
print(Person2)

   id             email
0   1  john@example.com
1   2   bob@example.com
   id             email
0   1  john@example.com
1   2   bob@example.com
2   3  john@example.com
